## Next Word Prediction use LSTM
Data: https://www.kaggle.com/datasets/ronikdedhia/next-word-prediction/data 

Task: Based on flare text data, build an LSTM model to predict sequence text: STM input data
1. Complete data preprocessing and convert text sequence data into data that can be used for separation operations
2. Check the data structure after text data preprocessing and perform data
ralia."), predict its corresponding follow-up
3. For string input (" flare is a teacher inai industry. He obtained his phd in Aus characters

Use the first 20 characters each time to predict the 21st character Note: Model structure: single-layer LSTM, output has 20 neurons;

In [2]:
data = open("1661-0.txt").read()
data = data.replace("\n", " ").replace("\r", " ")
print(data)

 Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle  This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.net   Title: The Adventures of Sherlock Holmes  Author: Arthur Conan Doyle  Release Date: November 29, 2002 [EBook #1661] Last Updated: May 20, 2019  Language: English  Character set encoding: UTF-8  *** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***    Produced by an anonymous Project Gutenberg volunteer and Jose Menendez    cover    The Adventures of Sherlock Holmes    by Arthur Conan Doyle    Contents      I.     A Scandal in Bohemia    II.    The Red-Headed League    III.   A Case of Identity    IV.    The Boscombe Valley Mystery    V.     The Five Orange Pips    VI.    The Man with the Twisted Lip    VII.   The Adventure of the Blue C

In [5]:
# 
letters = list(set(data))
print(letters)
num_letters = len(letters)
print(num_letters)

['’', 'é', '½', '6', '“', '£', 'k', 'l', '(', 'T', 'K', 'Z', 'r', 's', '$', 'd', '-', '?', 't', 'g', '#', '8', '5', 'W', 'h', '!', '_', 'o', 'a', 'Y', 'e', '\ufeff', 'E', 'æ', 'y', '*', '&', ',', '"', 'P', '2', 'p', '3', 'w', 'x', 'u', '”', 'c', 'q', ';', 'C', 'I', '%', 'b', 'à', 'N', ']', 'O', '—', 'f', 'i', ':', 'F', 'U', "'", '‘', 'n', ')', 'A', 'z', 'è', ' ', '/', 'M', 'H', 'œ', 'G', 'm', 'J', 'D', 'S', '[', 'L', '1', '9', '@', '.', '0', 'v', '7', 'j', 'B', 'X', '4', 'Q', 'â', 'V', 'R']
98


In [8]:
#create the dictionary
# int to char
int_to_char = {a:b for a,b in enumerate(letters)}
print(int_to_char)
#char to int
char_to_int = {b:a for a,b in enumerate(letters)}
print(char_to_int)


{0: '’', 1: 'é', 2: '½', 3: '6', 4: '“', 5: '£', 6: 'k', 7: 'l', 8: '(', 9: 'T', 10: 'K', 11: 'Z', 12: 'r', 13: 's', 14: '$', 15: 'd', 16: '-', 17: '?', 18: 't', 19: 'g', 20: '#', 21: '8', 22: '5', 23: 'W', 24: 'h', 25: '!', 26: '_', 27: 'o', 28: 'a', 29: 'Y', 30: 'e', 31: '\ufeff', 32: 'E', 33: 'æ', 34: 'y', 35: '*', 36: '&', 37: ',', 38: '"', 39: 'P', 40: '2', 41: 'p', 42: '3', 43: 'w', 44: 'x', 45: 'u', 46: '”', 47: 'c', 48: 'q', 49: ';', 50: 'C', 51: 'I', 52: '%', 53: 'b', 54: 'à', 55: 'N', 56: ']', 57: 'O', 58: '—', 59: 'f', 60: 'i', 61: ':', 62: 'F', 63: 'U', 64: "'", 65: '‘', 66: 'n', 67: ')', 68: 'A', 69: 'z', 70: 'è', 71: ' ', 72: '/', 73: 'M', 74: 'H', 75: 'œ', 76: 'G', 77: 'm', 78: 'J', 79: 'D', 80: 'S', 81: '[', 82: 'L', 83: '1', 84: '9', 85: '@', 86: '.', 87: '0', 88: 'v', 89: '7', 90: 'j', 91: 'B', 92: 'X', 93: '4', 94: 'Q', 95: 'â', 96: 'V', 97: 'R'}
{'’': 0, 'é': 1, '½': 2, '6': 3, '“': 4, '£': 5, 'k': 6, 'l': 7, '(': 8, 'T': 9, 'K': 10, 'Z': 11, 'r': 12, 's': 13, '$': 

In [9]:
#time_setp
time_setp = 20

In [10]:
import numpy as np
from keras.utils import to_categorical

# 滑动窗口提取数据
def extract_data(data, slide):
    x = []
    y = []
    for i in range(len(data) - slide):
        x.append([a for a in data[i:i + slide]])
        y.append(data[i + slide])
    return x, y

# 字符到数字的批量转化
def char_to_int_Data(x, y, char_to_int):
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append(char_to_int[y[i]])  # y[i] is a single character, not a list
    return x_to_int, y_to_int

# 实现输入字符文章的批量处理，输入整个字符、滑动窗口大小、转化字典
def data_preprocessing(data, slide, num_letters, char_to_int):
    # 提取数据
    char_Data = extract_data(data, slide)
    int_Data = char_to_int_Data(char_Data[0], char_Data[1], char_to_int)

    Input = int_Data[0]
    Output = list(np.array(int_Data[1]).flatten())

    # Reshape Input
    Input_RESHAPED = np.array(Input).reshape(len(Input), slide)

    # 创建随机矩阵
    new = np.random.randint(0, 10, size=(Input_RESHAPED.shape[0], Input_RESHAPED.shape[1], num_letters))

    # One-hot 编码
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i, j, :] = to_categorical(Input_RESHAPED[i, j], num_classes=num_letters)

    return new, Output


In [12]:
#extract X and y from text data 
X, y = data_preprocessing(data, time_setp, num_letters, char_to_int)
y = np.array(y) 
print(X.shape)
print(y.shape)

(581868, 20, 98)
(581868,)


In [13]:
print(X)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [14]:
print(y)

[13 71  9 ... 71 71 71]


In [15]:
#split the data 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, len(y_train))

(465494, 20, 98) 465494


In [16]:
y_train_category = to_categorical(y_train, num_letters)
print(y_train_category)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
#set up the model
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(units=20,input_shape=(X_train.shape[1], X_train.shape[2]),activation='relu'))
model.add(Dense(units=num_letters, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 20)                9520      
                                                                 
 dense_2 (Dense)             (None, 98)                2058      
                                                                 
Total params: 11,578
Trainable params: 11,578
Non-trainable params: 0
_________________________________________________________________


In [43]:
#train the model
model.fit(X_train, y_train_category, batch_size=1000,epochs=10)

Epoch 1/10
466/466 [==============================] - 11s 24ms/step - loss: 1.8295 - accuracy: 0.4617
Epoch 2/10
466/466 [==============================] - 12s 26ms/step - loss: 1.8268 - accuracy: 0.4629
Epoch 3/10
466/466 [==============================] - 11s 25ms/step - loss: 1.8267 - accuracy: 0.4626
Epoch 4/10
466/466 [==============================] - 11s 24ms/step - loss: 1.8240 - accuracy: 0.4634
Epoch 5/10
466/466 [==============================] - 12s 25ms/step - loss: 1.8231 - accuracy: 0.4631
Epoch 6/10
466/466 [==============================] - 12s 26ms/step - loss: 1.8212 - accuracy: 0.4636
Epoch 7/10
466/466 [==============================] - 12s 26ms/step - loss: 1.8184 - accuracy: 0.4644
Epoch 8/10
466/466 [==============================] - 10s 22ms/step - loss: 1.8185 - accuracy: 0.4652
Epoch 9/10
466/466 [==============================] - 11s 23ms/step - loss: 1.8159 - accuracy: 0.4655
Epoch 10/10
466/466 [==============================] - 11s 24ms/step - loss: 1.814

In [44]:
# Make prediction based on the training data
y_train_predict = np.argmax(model.predict(X_train), axis=-1)

print(y_train_predict)


14547/14547 [==============================] - 13s 918us/step
[ 4 71 24 ... 27 12 71]


In [46]:
#transform the int to letters
y_train_predict_char = [int_to_char[i] for i in y_train_predict]

print(y_train_predict_char)

['“', ' ', 'h', 'r', 'e', 'e', 'a', 'n', 'o', 'r', 'n', 'a', 'g', 'a', 'o', '.', ' ', 'n', 't', 'n', 'e', ' ', ' ', 'r', ' ', 'e', 'i', 'x', ' ', 's', 't', 't', 'h', 'o', 'e', 't', 'h', ' ', ' ', 'w', 'o', 't', 'u', 'd', 'e', 'd', 's', 't', ' ', 'a', 't', 'n', 'y', 'e', ' ', ' ', 'f', 't', 'n', 'I', 'e', ' ', 's', 'i', 't', 'o', 'm', 'a', 't', ' ', 'w', ' ', ' ', 'h', 'I', ' ', 'g', 's', 'n', 'h', 'a', 'I', 'x', 'I', 'n', ' ', ' ', 'n', 'o', 's', ' ', 'l', 'r', 'k', ' ', 'I', ' ', ' ', ' ', 'e', 't', 'a', 'e', 'r', ' ', 'h', 'b', ' ', 'h', ' ', 'n', 'e', ' ', ' ', 'h', ' ', 'a', 'a', ' ', 'e', ' ', 'a', 't', 's', 'a', 'o', 's', 'o', ' ', 'I', ' ', ' ', 't', 's', ' ', 'a', 'n', '0', 'I', 'r', 'n', 'e', 'a', 'e', ' ', ' ', 'n', 't', ' ', 'm', 'n', 'r', 'e', 'n', 'r', 's', 'e', 'e', 'h', 't', 'd', 't', 'o', ' ', ' ', 'g', ' ', 'u', 'r', 's', 'n', 'e', 't', ' ', 'r', 't', 'd', ' ', 't', 'l', 'g', 'e', 'e', 't', 'i', 't', ' ', 'x', 'e', ' ', 'a', 'a', ' ', ' ', 'h', ' ', 'o', ' ', 'n', 's',

In [47]:
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(y_train, y_train_predict)
print(accuracy_train)


0.46701568656094383


In [48]:
import numpy as np
from sklearn.metrics import accuracy_score

# Make predictions using model.predict
y_test_predict = np.argmax(model.predict(X_test), axis=-1)

# Calculate accuracy
accuracy_test = accuracy_score(y_test, y_test_predict)

print("Accuracy on test data:", accuracy_test)
print("Predicted values:", y_test_predict)
print("True values:", y_test)


3637/3637 [==============================] - 3s 909us/step
Accuracy on test data: 0.4646656469658171
Predicted values: [60 30 60 ...  7 30 71]
True values: [60 71 27 ... 15 30 47]


In [49]:
new_letters = 'I had seen little of Holmes lately. My marriage had drifted us away from each other.'

# Make predictions using model.predict
y_new_predict = np.argmax(model.predict(X_new), axis=-1)

# Convert the predicted integers to characters
y_new_predict_char = [int_to_char[i] for i in y_new_predict]

# Print the predicted characters
print(y_new_predict_char)


2/2 [==============================] - 0s 2ms/step
[' ', 't', 'o', 'l', 'm', 'e', 's', ' ', 'a', 'a', 'i', 'e', 'd', 'y', ' ', ' ', ' ', 'y', ' ', 's', 'y', 'y', 'e', 'e', 'n', 'n', 'e', ' ', 't', 'i', 'v', ' ', 't', 'o', 'e', 'g', 'o', 'e', 'r', ' ', 't', 'p', ' ', 'a', ' ', ' ', 'r', ' ', 't', 'o', 'o', 'm', ' ', 't', 'v', 'r', 'e', ' ', 't', 'f', ' ', 'e', 'r', ' ']


In [50]:
for i in range(0, X_new.shape[0]-20):
    print(new_letters[i:i+20],'--Next i--',  y_new_predict_char[i])

I had seen little of --Next i--  
 had seen little of  --Next i-- t
had seen little of H --Next i-- o
ad seen little of Ho --Next i-- l
d seen little of Hol --Next i-- m
 seen little of Holm --Next i-- e
seen little of Holme --Next i-- s
een little of Holmes --Next i--  
en little of Holmes  --Next i-- a
n little of Holmes l --Next i-- a
 little of Holmes la --Next i-- i
little of Holmes lat --Next i-- e
ittle of Holmes late --Next i-- d
ttle of Holmes latel --Next i-- y
tle of Holmes lately --Next i--  
le of Holmes lately. --Next i--  
e of Holmes lately.  --Next i--  
 of Holmes lately. M --Next i-- y
of Holmes lately. My --Next i--  
f Holmes lately. My  --Next i-- s
 Holmes lately. My m --Next i-- y
Holmes lately. My ma --Next i-- y
olmes lately. My mar --Next i-- e
lmes lately. My marr --Next i-- e
mes lately. My marri --Next i-- n
es lately. My marria --Next i-- n
s lately. My marriag --Next i-- e
 lately. My marriage --Next i--  
lately. My marriage  --Next i-- t
ately. My marr

In [ ]:
import numpy as np
from keras.utils import to categorical
#滑动窗口提取数据
def extract_data(data, slide):
    x = []
    y = []
    for i in range(len(data)- slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])
    return x,y
#字符到数字的批量转化
def char_to_int_Data(x,y, char_to_int):
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])
    return  x_to_int,  y_to_int
#实现输入字符文章的批量处理，输入整个字符、滑动窗口大小、转化字典
def data_preprocessing(data, slide, num letters, char_to_int):
    char_Data = extract_data(data, slide)
    int_Data = char_to_int_Data(char_Data[o], char_Data[1], char_toi_nt)
    Input = int_Data[0]
    Output = list(np.array(int_Data[1]).flatten())
    Input_RESHAPED = np.array(Input).reshape(len(input, slide))
    new = np.random.randint(0,10,size=Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters)
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:] = to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new, Output



LSTM text generation practice summary:
1. By building an LSTM model, the character generation function based on text sequence is realized:
2. Text loading and dictionary generation methods are learned:
3. Text data preprocessing methods are mastered, and the structure of conversion data is familiar;
4. Character prediction of new text data is realized

learning video:
https://www.bilibili.com/video/BV1JP4y117mX?spm_id_from=333.788.videopod.episodes&vd_source=6d5e118f93a0d98219382c6023b9dc07&p=6 